# Segmenting and Clustering Neighborhoods in Toronto
Created by Brandon Bellanti | Last updated on 2021-03-26

---

## Load Libraries

In [1]:
import pandas as pd, numpy as np
from bs4 import BeautifulSoup
import requests
import re

## Fetch Toronto neighborhood data from Wikipedia

**[Toronto Postal Codes](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)** on Wikipedia

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
S = requests.session()
r = S.get(wiki_url)
if r.status_code == 200:
    html = r.text
    print('Request successful')

Request successful


In [3]:
# print partial HTML string to verify
html[0:500]

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"8db48afd-5513-4ef4-835'

In [4]:
# create a Beautiful Soup object
soup = BeautifulSoup(html)

## Parse HTML table data

In [5]:
# search for all tables in the HTML and select the first one
tables = soup.find_all('table')
table = tables[0]

In [29]:
# create df to store neighborhood data
neighborhoods_df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhoods'])

Each cell text in the table contains a postal code, a borough name, and neighborhood name/s. The postal code is the first three characters, so I slice the string to return that value. The nieghborhoods and boroughs are consistently formatted as `Borough(Neighborhood)` – or `Borough(Neighborhood / Neighborhood / Neighborhood)` if there are multiple neighborhoods in the same borough. For example:
    
`M3ANorth York(Parkwoods)`

It's rare, but there are some postal codes that encompass multiple boroughs, such as:
    
`M3CNorth York(Don Mills)South(Flemingdon Park)`

I split the non-postal code string (from the fourth character to the end) on a closing parentheses ( ")" ) first, then iterate through the list created in case there are multiple boroughs. Then, for each neighborhood string, I replace any slashes ( "/" ) with commas so the neighborhoods are dilimited.

According to the project instructions, if a borough does not have an assigned neighborhood name, the neighborhood name should be the same as the borough.

In [30]:
for data in table.find_all('td'):
    text = data.text
    postal_code = text[:4].strip('\n')
    divs = text[4:].rstrip('\n\n').split(')')
    for div in divs:
        if div not in ['','Not assigned']:
            borough_town = div.split('(')
            borough = borough_town[0]
            try:
                neighborhoods = borough_town[1]
            except:
                neighborhoods = borough
            neighborhoods = neighborhoods.replace(' / ',', ')
                
            neighborhoods_df = neighborhoods_df.append(dict(
                                    PostalCode=postal_code,
                                    Borough=borough,
                                    Neighborhoods=neighborhoods),ignore_index=True)

neighborhoods_df.head()

,PostalCode,Borough,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [41]:
neighborhoods_df.shape

(112, 3)

Note, the call above to the `.shape` method does not account for the few instances where a postal code is repeated. The reason for these duplicates is that there are some postal codes that encompass multiple boroughs (like I mentioned before). The cell below shows the number of unique postal codes.

In [45]:
neighborhoods_df.drop_duplicates('PostalCode').shape # or neighborhoods_df['PostalCode'].nunique()

(103, 3)

## Join geographical data

In [32]:
# load spatial data
geo_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [46]:
# merge neighborhoods and spatial dataframes by the postal codes
toronto_df = neighborhoods_df.merge(geo_df, left_on='PostalCode',right_on='Postal Code',how='outer').drop(columns='Postal Code')
toronto_df.head(30)

,PostalCode,Borough,Neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M3B,North,North,43.745906,-79.352188
9,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


In [34]:
toronto_df.shape

(112, 5)

# Segmenting and clustering neighborhoods in Toronto

In [47]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [48]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [78]:
# filter toronto dataframe to include only boroughs that contain the word Toronto
toronto_data = toronto_df.loc[toronto_df['Borough'].str.contains('Toronto')].copy()

In [54]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhoods']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [55]:
CLIENT_ID = 'PHPYTVIC22RACPESMK2BTWWGZQLEP1H5X4OM5YD3UWPEQ1MN' # your Foursquare ID
CLIENT_SECRET = 'CKKXK00Y4X4IEBXB0BXABSPIQWRTTXG0CLII4E1RRNVU4ZMN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PHPYTVIC22RACPESMK2BTWWGZQLEP1H5X4OM5YD3UWPEQ1MN
CLIENT_SECRET:CKKXK00Y4X4IEBXB0BXABSPIQWRTTXG0CLII4E1RRNVU4ZMN


In [57]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
# explode dataframe to search each nieghborhood
toronto_data['Neighborhoods_list'] = toronto_data['Neighborhoods'].apply(lambda x: x.split(', '))
toronto_exploded = toronto_data.explode('Neighborhoods_list').reset_index(drop=True).rename(columns={'Neighborhoods_list':'Neighborhood'})
toronto_exploded.head(10)

,PostalCode,Borough,Neighborhoods,Latitude,Longitude,Neighborhood
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Regent Park
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Harbourfront
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Garden District
3,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Ryerson
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,St. James Town
5,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Beaches
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,Berczy Park
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,Central Bay Street
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564,Christie
9,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,Richmond


In [89]:
toronto_venues = getNearbyVenues(names=toronto_exploded['Neighborhood'],
                                   latitudes=toronto_exploded['Latitude'],
                                   longitudes=toronto_exploded['Longitude']
                                  )

Regent Park
Harbourfront
Garden District
Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond
Adelaide
King
Dufferin
Dovercourt Village
The Danforth  East
Harbourfront East
Union Station
Toronto Islands
Little Portugal
Trinity
The Danforth West
Riverdale
Toronto Dominion Centre
Design Exchange
Brockton
Parkdale Village
Exhibition Place
India Bazaar
The Beaches West
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
University of Toronto
Harbord
Runnymede
Swansea
Moore Park
Summerhill East
Kensington Market
Chinatown
Grange Park
Summerhill West
Rathnelly
South Hill
Forest Hill SE
Deer Park
CN Tower
King and Spadina
Railway Lands
Harbourfront West
Bathurst Quay
South Niagara
Island airport
Rosedale
Enclave of M5E
St. James Town
Cabbagetown
First Canadian Place
Underground city
Church a

In [100]:
print(f"""There were {toronto_venues.shape[0]} venues in {len(toronto_venues['Venue Category'].unique())} categories returned!""")

There were 3062 venues in 229 categories returned!


In [98]:
# most common venues
toronto_venues['Venue Category'].value_counts().head()

Coffee Shop           285
Café                  166
Restaurant            100
Hotel                  71
Italian Restaurant     71
Name: Venue Category, dtype: int64

In [106]:
# venues by neighborhood
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Adelaide                     92
Bathurst Quay                15
Berczy Park                  58
Brockton                     22
CN Tower                     15
Cabbagetown                  48
Central Bay Street           64
Chinatown                    66
Christie                     16
Church and Wellesley         75
Commerce Court              100
Davisville                   34
Davisville North              9
Deer Park                    14
Design Exchange             100
Dovercourt Village           15
Dufferin                     15
Enclave of M4L               16
Enclave of M5E               98
Exhibition Place             22
First Canadian Place        100
Forest Hill North & West      4
Forest Hill SE               14
Garden District             100
Grange Park                  66
Harbord                      33
Harbourfront                 45
Harbourfront East           100
Harbourfront West            15
High Park                    26
India Bazaar               

In [109]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [111]:
toronto_onehot.shape

(3062, 229)

In [113]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Adelaide,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.01087,0.0,0.0,0.010870,0.01087,0.0,0.01087,0.0,0.0,0.0,0.0,0.000000,0.010870,0.01087,0.010870,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.021739,0.0,0.01087,0.010870,0.0,0.0,0.01087,0.01087,0.021739,0.0,0.0,0.054348,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.032609,0.000000,0.108696,0.0,0.0,0.0,0.01087,0.000000,0.0,0.021739,0.000000,0.021739,0.0,0.000000,0.0,0.01087,0.0,0.032609,0.010870,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.01087,0.0,0.0,0.01087,0.0,0.000000,0.01087,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01087,0.0,0.0,0.0,0.0,0.01087,0.01087,0.000000,0.000000,0.000000

In [115]:
toronto_grouped.shape

(73, 229)

In [118]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.11
1             Café  0.05
2       Restaurant  0.04
3   Clothing Store  0.03
4  Thai Restaurant  0.03


----Bathurst Quay----
                 venue  freq
0       Airport Lounge  0.13
1      Airport Service  0.13
2     Airport Terminal  0.13
3                Plane  0.07
4  Rental Car Location  0.07


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2      Pharmacy  0.03
3   Cheese Shop  0.03
4        Bakery  0.03


----Brockton----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3              Bakery  0.05
4  Italian Restaurant  0.05


----CN Tower----
                 venue  freq
0       Airport Lounge  0.13
1      Airport Service  0.13
2     Airport Terminal  0.13
3                Plane  0.07
4  Rental Car Location  0.07


----Cabbagetown----
                venue  freq
0                Café  0.06
1         Coffee Shop  

In [119]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [125]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Clothing Store,Thai Restaurant,Salad Place,Cosmetics Shop,Hotel
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Rental Car Location,Coffee Shop,Plane,Bar,Sculpture Garden
2,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Restaurant,Pharmacy,Cheese Shop,Beer Bar,Japanese Restaurant
3,Brockton,Café,Coffee Shop,Breakfast Spot,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
4,CN Tower,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Rental Car Location,Coffee Shop,Plane,Bar,Sculpture Garden


In [126]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 4, 1, 1, 1, 1, 1], dtype=int32)

In [127]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_exploded

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhoods,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Regent Park,1,Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Theater,Yoga Studio,Dessert Shop,Shoe Store
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Harbourfront,1,Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Theater,Yoga Studio,Dessert Shop,Shoe Store
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Garden District,1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Fast Food Restaurant,Hotel,Electronics Store
3,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Ryerson,1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Fast Food Restaurant,Hotel,Electronics Store
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,St. James Town,1,Café,Coffee Shop,Park,Italian Restaurant,Restaurant,Gastropub,Beer Bar,Cocktail Bar,American Restaurant,Farmers Market


In [128]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1


In [129]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Central Toronto,Summerhill West,0,Coffee Shop,Pub,Liquor Store,Restaurant,Bank,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station
55,Central Toronto,Rathnelly,0,Coffee Shop,Pub,Liquor Store,Restaurant,Bank,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station
56,Central Toronto,South Hill,0,Coffee Shop,Pub,Liquor Store,Restaurant,Bank,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station
57,Central Toronto,Forest Hill SE,0,Coffee Shop,Pub,Liquor Store,Restaurant,Bank,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station
58,Central Toronto,Deer Park,0,Coffee Shop,Pub,Liquor Store,Restaurant,Bank,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station


#### Cluster 2


In [130]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Regent Park,1,Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Theater,Yoga Studio,Dessert Shop,Shoe Store
1,Downtown Toronto,Harbourfront,1,Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Theater,Yoga Studio,Dessert Shop,Shoe Store
2,Downtown Toronto,Garden District,1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Fast Food Restaurant,Hotel,Electronics Store
3,Downtown Toronto,Ryerson,1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Fast Food Restaurant,Hotel,Electronics Store
4,Downtown Toronto,St. James Town,1,Café,Coffee Shop,Park,Italian Restaurant,Restaurant,Gastropub,Beer Bar,Cocktail Bar,American Restaurant,Farmers Market
5,East Toronto,The Beaches,1,Asian Restaurant,Health Food Store,Trail,Pub,Wings Joint,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
6,Downtown Toronto,Berczy Park,1,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Restaurant,Pharmacy,Cheese Shop,Beer Bar,Japanese Restaurant
7,Downtown Toronto,Central Bay Street,1,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Portuguese Restaurant,Bookstore,Japanese Restaurant
8,Downtown Toronto,Christie,1,Grocery Store,Café,Park,Nightclub,Candy Store,Athletics & Sports,Italian Restaurant,Restaurant,Baby Store,Coffee Shop
9,Downtown Toronto,Richmond,1,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Clothing Store,Thai Restaurant,Salad Place,Cosmetics Shop,Hotel


#### Cluster 3


In [131]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Central Toronto,Roselawn,2,Garden,Wings Joint,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 4


In [132]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,East YorkEast Toronto,The Danforth East,3,Park,Convenience Store,Wings Joint,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
32,Central Toronto,Lawrence Park,3,Park,Swim School,Bus Line,Wings Joint,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
35,Central Toronto,Forest Hill North & West,3,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
49,Central Toronto,Moore Park,3,Park,Playground,Trail,Restaurant,Comic Shop,Concert Hall,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
50,Central Toronto,Summerhill East,3,Park,Playground,Trail,Restaurant,Comic Shop,Concert Hall,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
66,Downtown Toronto,Rosedale,3,Park,Playground,Trail,Wings Joint,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 5


In [133]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,CN Tower,4,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Rental Car Location,Coffee Shop,Plane,Bar,Sculpture Garden
60,Downtown Toronto,King and Spadina,4,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Rental Car Location,Coffee Shop,Plane,Bar,Sculpture Garden
61,Downtown Toronto,Railway Lands,4,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Rental Car Location,Coffee Shop,Plane,Bar,Sculpture Garden
62,Downtown Toronto,Harbourfront West,4,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Rental Car Location,Coffee Shop,Plane,Bar,Sculpture Garden
63,Downtown Toronto,Bathurst Quay,4,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Rental Car Location,Coffee Shop,Plane,Bar,Sculpture Garden
64,Downtown Toronto,South Niagara,4,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Rental Car Location,Coffee Shop,Plane,Bar,Sculpture Garden
65,Downtown Toronto,Island airport,4,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Rental Car Location,Coffee Shop,Plane,Bar,Sculpture Garden
